# **Scraping UFC Betting Data From BestFightOdds**

Data consists of the final quoted odds on UFC fights for each betting site listed on bestfightodds.com.
All bet types are included

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib.parse
import re
import urllib.request as urllib2

### **Get BestFightOdds URL for Each UFC Event in Wikipedia**

In [2]:
# pull list of past ufc events from wikipedia
url = "https://en.wikipedia.org/wiki/List_of_UFC_events"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
rows = soup.find('table', id="Past_events").find_all('tr')
columns = [x.text.strip() for x in rows[0].find_all('th')]
past_events = pd.DataFrame([[x.text.strip() for x in row.find_all('td')] for row in rows[1:]], columns = columns)
urls = []
for row in rows[1:]:
    s = row.find_all('td')
    if  len(s) >= 1 and getattr(s[1], 'a', None) != None:
       urls.append('https://en.wikipedia.org' + s[1].a['href'].strip())
    else:
        urls.append(None)
past_events['url'] = urls
past_events = past_events[past_events['Attendance'] != 'Cancelled']
past_events.sort_values('#', ascending = False, inplace = True)
past_events['Date'] = pd.to_datetime(past_events['Date'])
del past_events['Ref.']
#past_events.to_csv("data/wikipedia_all_ufc_events.csv", index = False)

In [3]:
events = past_events.copy()

In [4]:
events.head()

,#,Event,Date,Venue,Location,Attendance,url
520,—,UFC 151: Jones vs. Henderson,2012-09-01,Mandalay Bay Events Center,"Las Vegas, Nevada, U.S.",Canceled,https://en.wikipedia.org/wiki/UFC_151
449,—,UFC 176: Aldo vs. Mendes II,2014-08-02,Staples Center,"Los Angeles, California, U.S.",Canceled,https://en.wikipedia.org/wiki/UFC_176
357,—,UFC Fight Night: Lamas vs. Penn,2016-10-15,Mall of Asia Arena,"Pasay, Philippines",Canceled,https://en.wikipedia.org/wiki/UFC_Fight_Night:...
267,—,UFC 233: Cejudo vs. Dillashaw,2019-01-26,Honda Center,"Anaheim, California, U.S.",Canceled,https://en.wikipedia.org/wiki/UFC_233
217,—,UFC Fight Night: Woodley vs. Edwards,2020-03-21,The O2 Arena,"London, England, UK",None,https://en.wikipedia.org/wiki/UFC_Fight_Night:...


In [ ]:
#events = pd.read_csv('data/wikipedia_all_ufc_events.csv')
search = events['Event'].apply(lambda x: 'https://www.bestfightodds.com/search?query=' + urllib.parse.quote(x))
events['Date'] = pd.to_datetime(events['Date'])
fightodds_url = [''] * events.shape[0]

for i in range(events.shape[0]):  
    date_of_card = events.iloc[i]['Date']
    page = requests.get(search.iloc[i])
    soup = BeautifulSoup(page.text, 'lxml')
    print(f"Processing event: {events['Event'][i]}")

    # skip if no possible matches found
    if soup.find('table') == None or page.text.lower().find("No betting lines available for this event".lower()) != -1:
        fightodds_url[i] = ''
        print(f"No possible matches found for event: {events['Event'][i]}")
        continue 
    
    # if query redirects to exact match take query as url
    if soup.find('tr', class_ = 'even')  != None:
        fightodds_url[i] = search.iloc[i]
        print(f"Found exact match for event: {events['Event'][i]}")
        continue
        
    table = soup.find_all('table', class_ = 'content-list') [-1].find_all('tr')
    final_url = ''
    for p in table:
        if pd.to_datetime(p.find('td').text) == date_of_card:
            final_url = 'https://www.bestfightodds.com'+ p.find_all('td')[1].a['href']
            fightodds_url[i] = final_url
            print(f"Found match for event: {events['Event'][i]}")
            break
    
    # in some cases time zones cause dates to be off by a day. 
    if final_url == '':
        for p in table:
            if np.abs((pd.to_datetime(p.find('td').text) - date_of_card) / pd.to_timedelta(1, unit='D')) <= 1 :
                final_url = 'https://www.bestfightodds.com'+ p.find_all('td')[1].a['href']
                fightodds_url[i] = final_url
                print(f"Found match for event: {events['Event'][i]}")
                break

    
            
events['fight_odds_url'] =  fightodds_url

events.head()

Processing event: UFC Fight Night: Edwards vs. Brady
No possible matches found for event: UFC Fight Night: Edwards vs. Brady
Processing event: UFC Fight Night: Vettori vs. Dolidze 2
No possible matches found for event: UFC Fight Night: Vettori vs. Dolidze 2
Processing event: UFC 313: Pereira vs. Ankalaev
No possible matches found for event: UFC 313: Pereira vs. Ankalaev
Processing event: UFC Fight Night: Kape vs. Almabayev
No possible matches found for event: UFC Fight Night: Kape vs. Almabayev
Processing event: UFC Fight Night: Cejudo vs. Song
No possible matches found for event: UFC Fight Night: Cejudo vs. Song
Processing event: UFC Fight Night: Cannonier vs. Rodrigues
No possible matches found for event: UFC Fight Night: Cannonier vs. Rodrigues
Processing event: UFC 312: du Plessis vs. Strickland 2
No possible matches found for event: UFC 312: du Plessis vs. Strickland 2
Processing event: UFC Fight Night: Adesanya vs. Imavov
No possible matches found for event: UFC Fight Night: Ades

In [3]:
# get coverage of how many ufc events in Wikipedia have bestfightodds urls
events['fight_odds_url'].groupby(events['Date'].apply(lambda x: x.year)).apply(lambda x: (x != '').sum())/events['fight_odds_url'].groupby(events['Date'].apply(lambda x: x.year)).size()

Date
1993    0.000000
1994    0.000000
1995    0.000000
1996    0.200000
1997    0.400000
1998    0.000000
1999    0.166667
2000    0.000000
2001    0.200000
2002    0.142857
2003    0.000000
2004    0.200000
2005    0.200000
2006    0.000000
2007    0.578947
2008    1.000000
2009    1.000000
2010    0.958333
2011    0.925926
2012    0.967742
2013    1.000000
2014    1.000000
2015    0.951220
2016    1.000000
2017    1.000000
2018    1.000000
2019    0.952381
2020    1.000000
Name: fight_odds_url, dtype: float64

In [4]:
# view missing events
pd.set_option('display.max_rows', 500)
events[(events['fight_odds_url'] == '') & ( events['Date'].apply(lambda x: x.year) >= 2008) ]

,#,Event,Date,Venue,Location,Attendance,url,fight_odds_url
11,502,UFC Fight Night: Błachowicz vs. Jacaré,2019-11-16,Ginásio do Ibirapuera,"São Paulo, Brazil","10,344",https://en.wikipedia.org/wiki/UFC_Fight_Night:...,
45,468,UFC Fight Night: Błachowicz vs. Santos,2019-02-23,O2 arena,"Prague, Czech Republic","16,583",https://en.wikipedia.org/wiki/UFC_Fight_Night:...,
171,342,The Ultimate Fighter: Team McGregor vs. Team F...,2015-12-11,The Chelsea at The Cosmopolitan,"Las Vegas, Nevada, U.S.","2,566",https://en.wikipedia.org/wiki/The_Ultimate_Fig...,
187,326,The Ultimate Fighter: American Top Team vs. Bl...,2015-07-12,MGM Grand Garden Arena,"Las Vegas, Nevada, U.S.","4,844",https://en.wikipedia.org/wiki/The_Ultimate_Fig...,
317,196,UFC on Fox: Evans vs. Davis,2012-01-28,United Center,"Chicago, Illinois, U.S.","16,963",https://en.wikipedia.org/wiki/UFC_on_Fox:_Evan...,
330,183,UFC Fight Night: Shields vs. Ellenberger,2011-09-17,Ernest N. Morial Convention Center,"New Orleans, Louisiana, U.S.","7,112",https://en.wikipedia.org/wiki/UFC_Fight_Night:...,
335,178,UFC Live: Kongo vs. Barry,2011-06-26,Consol Energy Center,"Pittsburgh, Pennsylvania, U.S.","7,792",https://en.wikipedia.org/wiki/UFC_Live:_Kongo_...,
357,156,UFC Live: Jones vs. Matyushenko,2010-08-01,San Diego Sports Arena,"San Diego, California, U.S.","8,132",https://en.wikipedia.org/wiki/UFC_Live:_Jones_...,


In [5]:
# fill in missing events manually
events.loc[events['Event'] == 'UFC Fight Night: Błachowicz vs. Jacaré', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-on-espn-22-blachowicz-vs-jacare-1755'
events.loc[events['Event'] == 'UFC Fight Night: Błachowicz vs. Santos', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-on-espn-3-blachowicz-vs-santos-1638'
events.loc[events['Event'] == 'UFC on Fox: Evans vs. Davis', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-on-fox-2-482'
events.loc[events['Event'] == 'UFC Fight Night: Shields vs. Ellenberger', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-fight-night-25-battle-on-the-bayou-406'
events.loc[events['Event'] == 'UFC Live: Kongo vs. Barry', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-on-versus-4-379'
events.loc[events['Event'] == 'UFC Live: Jones vs. Matyushenko', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-on-versus-2-281'
events.loc[events['Event'] == 'The Ultimate Fighter: Team McGregor vs. Team Faber Finale', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-the-ultimate-fighter-22-finale-edgar-vs-mendes-1011'
events.loc[events['Event'] == 'The Ultimate Fighter: Heavy Hitters Finale', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-the-ultimate-fighter-28-finale-dos-anjos-vs-usman-1586'
events.loc[events['Event'] == 'The Ultimate Fighter: American Top Team vs. Blackzilians Finale', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-the-ultimate-fighter-21-finale-ellenberger-vs-thompson-971'
events.loc[events['Event'] == 'The Ultimate Fighter: Undefeated Finale', 'fight_odds_url'] = 'https://www.bestfightodds.com/events/ufc-the-ultimate-fighter-27-finale-tavares-vs-adesanya-1503'

# get coverage of % ufc events are available
print(events['fight_odds_url'].groupby(events['Date'].apply(lambda x: x.year)).apply(lambda x: (x != '').sum())/events['fight_odds_url'].groupby(events['Date'].apply(lambda x: x.year)).size())
print(sum((events.Date  >= pd.to_datetime("Jan 1, 2008")) & (events.fight_odds_url.isna() | (events.fight_odds_url == ''))))
events = events[~(events.fight_odds_url.isna() | (events.fight_odds_url == ''))]
events.to_csv("../../data/bestfightodds_data/bestfightodds_urls.csv", index = False)


Date
1993    0.000000
1994    0.000000
1995    0.000000
1996    0.200000
1997    0.400000
1998    0.000000
1999    0.166667
2000    0.000000
2001    0.200000
2002    0.142857
2003    0.000000
2004    0.200000
2005    0.200000
2006    0.000000
2007    0.578947
2008    1.000000
2009    1.000000
2010    1.000000
2011    1.000000
2012    1.000000
2013    1.000000
2014    1.000000
2015    1.000000
2016    1.000000
2017    1.000000
2018    1.000000
2019    1.000000
2020    1.000000
Name: fight_odds_url, dtype: float64
0


# As a quality check for later pull all closing odds 

i.e. to make sure the final odds in the time series match the closing odds

In [11]:
# pull list of past ufc moneylines from bestfightodds
dfs = []
events = events.loc[(events['fight_odds_url'] != '') & 
                    (events['fight_odds_url'].notna()), :]
for card_index, url in enumerate(events['fight_odds_url']):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    columns = ['class']+[t.text for t in soup.find_all('table',class_ = 'odds-table')[1].thead.tr.find_all('th')][0:-1]
    rowtable = soup.find_all('table',class_ = 'odds-table')[1].tbody.find_all('tr')
    rows = [([row['class'][0]] + [row.find('th').text] \
             + [t.text.replace('▲','').replace('▼','') for t in row.find_all('td')])[0:-2] for row in rowtable]
    df = pd.DataFrame(rows, columns = columns)
    df.rename(columns={'':'Bet'}, inplace=True)
    fight1 = []
    fight2 = []
    fighter1 = ''
    fighter2 = ''
    
    hit_event_props = False
    for i in range(df.shape[0]):
        
        if df.iloc[i]['Bet'].lower() == 'event props':
            fighter1 = 'Event Prop'
            fighter2 = fighter1
            hit_event_props = True
            
        elif hit_event_props == False and df.iloc[i]['class'] == 'even':
            fighter1 = df.iloc[i]['Bet']
            fighter2 = df.iloc[i+1]['Bet']
            
        fight1.append(fighter1)
        fight2.append(fighter2)
        
    df.insert(0,'fighter1', fight1)
    df.insert(1,'fighter2', fight2)
    df.insert(2,'bet_type', np.where((df['class'] == 'even') | (df['class'] == 'odd'),'Ordinary','Prop'))
    df.loc[df['fighter1'] == 'Event Prop','bet_type'] = 'Event Prop'
    del df['class']
    # get rid of row showing event props if it's present
    if hit_event_props == True:
        df.drop(df[(df['Bet'].map(lambda x: x.lower()) == 'event props')].index, inplace=True)
    df['url'] = url
    df['Card_Date'] = events['Date'].iloc[card_index]
    df['Event'] = events['Event'].iloc[card_index]
    dfs.append(df)

In [12]:
all_odds = pd.concat(dfs)
all_odds.rename({'William\xa0H.':'William_H', 'SportsInt.':'SportsInt'}, inplace = True, axis = 1)
betting_sites= ['5Dimes','BetDSI','BookMaker','SportBet','Bet365','Bovada', 
                'Sportsbook','William_H','Pinnacle','SportsInt','BetOnline','Intertops']
for col in betting_sites:
    all_odds[col] = pd.to_numeric(all_odds[col], errors = 'coerce')
all_odds.to_csv("../../data/bestfightodds_data/moneyline_data_at_close.csv", index = False)